In [ ]:
from langchain_community.document_loaders import PyPDFLoader
 
loader = PyPDFLoader("./data/Uploads/Mutalib_2019_IOP_Conf._Ser.__Mater._Sci._Eng._469_012092.pdf")
data = loader.load()

print(data)

ModuleNotFoundError: No module named 'langchain_community'